In [14]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


###### ArCo ARMADO DE LA BASE#######
import os
from typing import List
from datetime import datetime

# Establecer el directorio de trabajo
os.chdir("C:/Users/supegui/Documents/Metodología ArCo/Python/Excel")
# Saca la lista de archivos del directorio de trabajo
#os.listdir()
# Obtener la fecha actual
fecha_actual = datetime.now().strftime("%Y %m %d")
#print(fecha_actual)

# Construir el nombre del archivo basado en la fecha actual
#nombre_archivo = f"{fecha_actual} Informe ArCo.xlsx"

# Construir el nombre del archivo cuando la fecha no es la actual. Aquí solo se modifica la fecha. 
nombre_archivo = "2024 11 01 - Base Original.xlsx"

data_arco = pd.read_excel(nombre_archivo)

In [15]:
data_arco.head()

,Código,Sector,Entidad,Nombre del Instrumento,Descripcion,Página Web,Antigüedad,Iteración,Emprendedores,Mipymes,...,1. Señale cuál o cuáles de las siguientes brechas,2. ¿En el diseño del instrumento se analizó el posible impacto diferenciado?,3. ¿El instrumento incorpora un enfoque interseccional?,Recursos fuente PGN,Recursos fuente SGR,Recursos fuente Cooperacion,Recursos fuente Esfuerzo Fiscal,Recursos fuente Parafiscales,Recursos fuente Otro,Estado
0,300,Trabajo,SERVICIO NACIONAL DE APRENDIZAJE - SENA,Programa de Formación Continua Especializada (...,El Programa de Formación Continua Especializad...,https://www.sena.edu.co/es-co/Empresarios/Pagi...,25.0,Primera iteración 2025 – Mes de reporte: Septi...,No,No,...,No,No,No,0,0,0,0,59708000000,0,Actualizado
1,301,Transporte,CORPORACION AUTONOMA REGIONAL DEL RIO GRANDE D...,Mantenimiento del Canal Navegable del Río Gran...,Cormagdalena adelantará d ragados y obras de d...,https://cormagdalena.gov.co/,28.0,Segunda iteración 2024 - Mes de reporte: Febre...,No,No,...,No,No,No,0,0,0,0,0,0,Desactualizado
2,310,Estadística,DEPARTAMENTO ADMINISTRATIVO NACIONAL DE ESTADI...,Información estadística de la temática económica,"El DANE ofrece diferentes investigaciones, vis...",https://www.dane.gov.co/index.php/estadisticas...,29.0,Primera iteración 2025 – Mes de reporte: Septi...,No,Si,...,No,No,No,40000000000,0,0,0,0,0,Actualizado
3,324,Hacienda,MINISTERIO DE HACIENDA Y CREDITO PUBLICO,Sistema Integrado de Información Financiera - ...,El Sistema Integrado de Información Financiera...,https://www.minhacienda.gov.co/webcenter/porta...,5.0,Primera iteración 2025 – Mes de reporte: Septi...,No,No,...,No,No,No,20713350446,0,0,0,0,0,Actualizado
4,343,Presidencia,AGENCIA PARA LA REINCORPORACION Y LA NORMALIZA...,Información del proceso de Reintegración y Rei...,"Brindar información de manera oportuna, veraz ...",http://www.reincorporacion.gov.co/es/la-reinte...,10.0,Primera iteración 2025 – Mes de reporte: Septi...,No,No,...,No,No,No,12525971945,0,0,0,0,0,Actualizado


In [16]:
# 1. Primero, creamos una matriz de características binarias para cada instrumento
def crear_matriz_caracteristicas(data_arco):
    # Seleccionamos las columnas relevantes

    # Tipos de usuario

    cols_usuarios = [
    #             "Emprendedores",
                 "Mipymes",
                 "Grandes empresas",
    #            "Gobierno ",
    #            "Academia",
    #            "Entidades de soporte",
    #            "Personas naturales"
    ]

    # Objetivos de Política

    cols_usuarios = [
                    "Innovación", "Emprendimiento", "Economía Popular",
                    "Transferencia de Conocimiento y Tecnología", "Investigación",
                    "Formación de Capital Humano", "Propiedad intelectual", "Calidad",
                    "Clúster o encadenamientos", "Financiación", "Comercialización",
                    "Formalización", "Crecimiento Sostenible", "Inclusión Financiera",
                    "Comercio Electrónico", "Educación económica y financiera", "Competencias Digitales"
    ]


    
    # Creamos una matriz donde cada fila es un instrumento y cada columna es un tipo de usuario
    matriz_caracteristicas = data_arco[cols_usuarios].copy()
    
    # Convertimos 'Si'/'No' a 1/0
    matriz_caracteristicas = (matriz_caracteristicas == 'Si').astype(int)
    
    return matriz_caracteristicas

In [22]:
matriz_cara = crear_matriz_caracteristicas(data_arco)

print(matriz_cara)

     Innovación  Emprendimiento  Economía Popular  \
0             0               0                 0   
1             0               0                 0   
2             0               0                 0   
3             1               0                 0   
4             0               0                 0   
..          ...             ...               ...   
414           0               0                 0   
415           0               0                 0   
416           0               0                 0   
417           1               0                 0   
418           0               0                 1   

     Transferencia de Conocimiento y Tecnología  Investigación  \
0                                             0              0   
1                                             0              0   
2                                             0              1   
3                                             0              0   
4                                

In [23]:
"""
# 2. Calculamos la similitud entre instrumentos - Jaccard
def calcular_similitud(data_arco):
    # Obtenemos la matriz de características
    matriz = crear_matriz_caracteristicas(data_arco)
    
    # Inicializamos la matriz de similitud
    n_instrumentos = len(data_arco)
    matriz_similitud = np.zeros((n_instrumentos, n_instrumentos))
    
    # Calculamos la similitud para cada par de instrumentos
    for i in range(n_instrumentos):
        for j in range(n_instrumentos):
            # Extraemos los vectores de características para los instrumentos i y j
            instrumento_i = matriz.iloc[i].values
            instrumento_j = matriz.iloc[j].values
            
            # Calculamos similitud usando coincidencias
            coincidencias = sum(instrumento_i & instrumento_j)  # AND lógico
            total_usuarios = sum(instrumento_i | instrumento_j)  # OR lógico
            
            # Calculamos el coeficiente de Jaccard
            if total_usuarios > 0:
                similitud = coincidencias / total_usuarios
            else:
                similitud = 0
                
            matriz_similitud[i][j] = similitud
    
    return matriz_similitud
"""


from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calcular_similitud(data_arco):
    # Obtenemos la matriz de características
    matriz = crear_matriz_caracteristicas(data_arco)
    
    # Calculamos la matriz de similitud de coseno
    matriz_similitud = cosine_similarity(matriz)
    
    return matriz_similitud


In [24]:
# 3. Convertimos la similitud a ratings
def calcular_ratings(data_arco):
    # Calculamos la matriz de similitud
    matriz_similitud = calcular_similitud(data_arco)
    
    # Creamos un DataFrame con los ratings
    ratings = []
    
    for i in range(len(matriz_similitud)):
        for j in range(len(matriz_similitud)):
            if i != j:  # No incluimos la similitud de un instrumento consigo mismo
                ratings.append({
                    'Instrumento_A': data_arco.iloc[i]['Código'],
                    'Nombre_A': data_arco.iloc[i]['Nombre del Instrumento'],
                    'Instrumento_B': data_arco.iloc[j]['Código'],
                    'Nombre_B': data_arco.iloc[j]['Nombre del Instrumento'],
                    'Rating': round(matriz_similitud[i][j] * 5, 2)  # Convertimos a escala 0-5
                })
    
    df_ratings = pd.DataFrame(ratings)
    
    # Ordenamos por rating de mayor a menor
    df_ratings = df_ratings.sort_values('Rating', ascending=False)
    
    return df_ratings


In [25]:

"""
def calcular_ratings(data_arco):
    # Calculamos la matriz de similitud
    matriz_similitud = calcular_similitud(data_arco)
    
    # Creamos un DataFrame con los ratings
    ratings = []
    
    for i in range(len(matriz_similitud)):
        for j in range(len(matriz_similitud)):
            # Solo procesamos cuando:
            # 1. No es el mismo instrumento (i != j)
            # 2. La similitud es 1 (mismo tipo de usuario)
            if i != j and matriz_similitud[i][j] == 1:
                ratings.append({
                    'Instrumento_A': data_arco.iloc[i]['Código'],
                    'Nombre_A': data_arco.iloc[i]['Nombre del Instrumento'],
                    'Instrumento_B': data_arco.iloc[j]['Código'],
                    'Nombre_B': data_arco.iloc[j]['Nombre del Instrumento'],
                    'Rating': 5.0  # Al ser similitud 1, el rating será 5
                })
    
    # Si no hay ratings, retornamos un DataFrame vacío con las columnas correctas
    if not ratings:
        return pd.DataFrame(columns=['Instrumento_A', 'Nombre_A', 'Instrumento_B', 'Nombre_B', 'Rating'])
    
    df_ratings = pd.DataFrame(ratings)
    
    # Ordenamos por rating de mayor a menor
    df_ratings = df_ratings.sort_values('Rating', ascending=False)
    
    return df_ratings
"""

"\ndef calcular_ratings(data_arco):\n    # Calculamos la matriz de similitud\n    matriz_similitud = calcular_similitud(data_arco)\n    \n    # Creamos un DataFrame con los ratings\n    ratings = []\n    \n    for i in range(len(matriz_similitud)):\n        for j in range(len(matriz_similitud)):\n            # Solo procesamos cuando:\n            # 1. No es el mismo instrumento (i != j)\n            # 2. La similitud es 1 (mismo tipo de usuario)\n            if i != j and matriz_similitud[i][j] == 1:\n                ratings.append({\n                    'Instrumento_A': data_arco.iloc[i]['Código'],\n                    'Nombre_A': data_arco.iloc[i]['Nombre del Instrumento'],\n                    'Instrumento_B': data_arco.iloc[j]['Código'],\n                    'Nombre_B': data_arco.iloc[j]['Nombre del Instrumento'],\n                    'Rating': 5.0  # Al ser similitud 1, el rating será 5\n                })\n    \n    # Si no hay ratings, retornamos un DataFrame vacío con las colum

In [27]:
# 4. Función principal que integra todo el proceso
def generar_ratings_colaborativos(data_arco):
    # Calculamos los ratings
    df_ratings = calcular_ratings(data_arco)
    
    # Filtramos solo los pares con rating significativo (por ejemplo, mayor a 2)
    df_ratings_significativos = df_ratings[df_ratings['Rating'] > 2]
    
    return df_ratings_significativos

# Ejemplo de uso:
ratings_finales = generar_ratings_colaborativos(data_arco)
#print(ratings_finales.head(1000))

In [9]:
print(ratings_finales)

        Instrumento_A                                           Nombre_A  \
123828           2805  Servicio de Distribución de Productos Especial...   
131403           2848  Participación en espacios de articulación en t...   
59836             811  Análisis del papel del servicio de transporte ...   
139104           2969  AMPLIACIÓN DEL CONOCIMIENTO DE LOS RECURSOS MI...   
135406           2935  Justamente - Aprendiendo sobre justicia y rest...   
...               ...                                                ...   
19539             498                                    Fondo Emprender   
130436           2839                Centros de Reindustralización ZASCA   
109409           2445               Corredor de financiamiento climático   
162078           3625    Promoción de empresas en reactivación económica   
117351           2557   Fortalecimiento Grupos de Ahorro y Crédito Local   

        Instrumento_B                                           Nombre_B  \
123828     

## Sistema de recomendación


In [28]:
from surprise import Dataset, Reader, SVD, KNNBasic, NMF, accuracy, similarities
from surprise.model_selection import train_test_split, cross_validate
from collections import defaultdict

class SistemaRecomendacion:
    def __init__(self, df_ratings):
        """
        Inicializa el sistema de recomendación con el DataFrame de ratings
        
        Parameters:
        -----------
        df_ratings : pandas.DataFrame
            DataFrame que debe contener las columnas ['Instrumento_A', 'Instrumento_B', 'Rating']
        """
        # Validar que el DataFrame tiene las columnas necesarias
        required_columns = ['Instrumento_A', 'Instrumento_B', 'Rating']
        if not all(col in ratings_finales.columns for col in required_columns):
            raise ValueError(f"El DataFrame debe contener las columnas: {required_columns}")
            
        self.ratings_finales = ratings_finales.copy()
        
        # Verificar que no hay valores nulos
        if self.ratings_finales[required_columns].isna().any().any():
            raise ValueError("El DataFrame contiene valores nulos")
        
        # Aseguramos que los ratings están en el rango correcto
        self.ratings_finales['Rating'] = self.ratings_finales['Rating'].clip(0, 5)
        
        # Configuramos el reader con un rango válido
        self.reader = Reader(rating_scale=(0, 5))
        
        try:
            # Convertimos a formato Surprise
            self.data = Dataset.load_from_df(
                self.ratings_finales[required_columns], 
                self.reader
            )
        except Exception as e:
            raise ValueError(f"Error al convertir datos al formato Surprise: {str(e)}")
        
        # Definimos los modelos base
        self.modelos = {
            'SVD': SVD(
                n_factors=50,
                n_epochs=20,
                lr_all=0.005,
                reg_all=0.02,
                random_state=42
            ),
            'NMF': NMF(
                n_factors=15,
                n_epochs=50,
                random_state=42
            )
        }
        
        # Calcular el número de ratings por instrumento de manera correcta
        ratings_a = self.ratings_finales['Instrumento_A'].value_counts()
        ratings_b = self.ratings_finales['Instrumento_B'].value_counts()
        
        # Combinar los conteos
        todos_instrumentos = set(ratings_a.index) | set(ratings_b.index)
        ratings_totales = pd.Series(0, index=todos_instrumentos)
        
        for instrumento in todos_instrumentos:
            total = (ratings_a.get(instrumento, 0) + 
                    ratings_b.get(instrumento, 0))
            ratings_totales[instrumento] = total
        
        min_ratings = ratings_totales.min()
        print(f"Número mínimo de ratings por instrumento: {min_ratings}")
        print(f"Número máximo de ratings por instrumento: {ratings_totales.max()}")
        print(f"Promedio de ratings por instrumento: {ratings_totales.mean():.2f}")
        
        # Agregamos KNN solo si tenemos suficientes datos y ratings
        if len(self.ratings_finales) > 10 and min_ratings >= 2:
            # Calculamos k de manera más conservadora
            k = min(
                20,  # máximo valor de k
                max(2, len(self.ratings_finales) // 10)  # k proporcional al tamaño del dataset
            )
            
            self.modelos['KNN'] = KNNBasic(
                k=k,
                min_k=2,
                sim_options={
                    'name': 'pearson_baseline',
                    'user_based': False,
                    'min_support': 2,
                    'shrinkage': 100
                }
            )
            print(f"KNN configurado con k={k}")
        else:
            print("No se incluirá KNN debido a insuficientes datos o ratings por instrumento")
        
        self.mejor_modelo = None
        self.metricas_modelos = {}
        
        print(f"\nTotal de instrumentos únicos: {len(todos_instrumentos)}")
        print(f"Total de ratings: {len(self.ratings_finales)}")

    def evaluar_modelos(self):
        """
        Evalúa diferentes modelos usando validación cruzada
        
        Returns:
        --------
        dict
            Diccionario con las métricas de evaluación de cada modelo
        """
        if len(self.ratings_finales) < 5:
            raise ValueError("Se necesitan al menos 5 ratings para evaluar los modelos")
            
        print("\nEvaluando modelos...")
        for nombre, modelo in self.modelos.items():
            try:
                # Realizar validación cruzada
                n_folds = min(5, len(self.ratings_finales) // 2)
                n_folds = max(2, n_folds)
                
                resultados = cross_validate(
                    modelo, 
                    self.data, 
                    measures=['RMSE', 'MAE'], 
                    cv=n_folds, 
                    verbose=False
                )
                
                self.metricas_modelos[nombre] = {
                    'RMSE_medio': np.mean(resultados['test_rmse']),
                    'MAE_medio': np.mean(resultados['test_mae']),
                    'tiempo_fit_medio': np.mean(resultados['fit_time']),
                    'tiempo_test_medio': np.mean(resultados['test_time'])
                }
                
                print(f"✓ Modelo {nombre} evaluado exitosamente")
                print(f"  RMSE: {self.metricas_modelos[nombre]['RMSE_medio']:.4f}")
                print(f"  MAE: {self.metricas_modelos[nombre]['MAE_medio']:.4f}")
                
            except Exception as e:
                print(f"✗ Error evaluando modelo {nombre}: {str(e)}")
                print("  Este modelo será excluido de las recomendaciones")
                continue
        
        if not self.metricas_modelos:
            raise ValueError("Ningún modelo pudo ser evaluado exitosamente")
            
        # Encontrar el mejor modelo basado en RMSE
        mejor_modelo_nombre = min(
            self.metricas_modelos, 
            key=lambda x: self.metricas_modelos[x]['RMSE_medio']
        )
        self.mejor_modelo = self.modelos[mejor_modelo_nombre]
        print(f"\n→ Mejor modelo: {mejor_modelo_nombre}")
        
        return self.metricas_modelos


    def entrenar_mejor_modelo(self):
        """
        Entrena el mejor modelo con todos los datos
        """
        if self.mejor_modelo is None:
            raise ValueError("Debe evaluar los modelos primero")
            
        trainset = self.data.build_full_trainset()
        self.mejor_modelo.fit(trainset)

    def generar_recomendaciones(self, instrumento_id, n=5):
        """
        Genera recomendaciones para un instrumento específico con manejo de errores
        """
        if self.mejor_modelo is None:
            raise ValueError("Debe entrenar el modelo primero")
            
        # Obtener todos los instrumentos únicos
        instrumentos_unicos = set(self.ratings_finales['Instrumento_A'].unique()) | \
                            set(self.ratings_finales['Instrumento_B'].unique())
        
        if instrumento_id not in instrumentos_unicos:
            raise ValueError(f"Instrumento {instrumento_id} no encontrado en el conjunto de datos")
        
        # Generar predicciones para todos los pares posibles
        predicciones = []
        for otro_instrumento in instrumentos_unicos:
            if otro_instrumento != instrumento_id:
                try:
                    pred = self.mejor_modelo.predict(instrumento_id, otro_instrumento)
                    predicciones.append({
                        'Instrumento_origen': instrumento_id,
                        'Instrumento_recomendado': otro_instrumento,
                        'Rating_predicho': pred.est,
                        'Detalles': pred.details
                    })
                except Exception as e:
                    print(f"Error prediciendo para {otro_instrumento}: {str(e)}")
                    continue
        
        if not predicciones:
            raise ValueError("No se pudieron generar predicciones")
            
        # Convertir a DataFrame y ordenar
        df_recomendaciones = pd.DataFrame(predicciones)
        df_recomendaciones = df_recomendaciones.sort_values(
            'Rating_predicho', 
            ascending=False
        ).head(n)
        
        return df_recomendaciones

    def mostrar_metricas(self):
        """
        Muestra las métricas de evaluación de todos los modelos
        """
        if not self.metricas_modelos:
            raise ValueError("No hay métricas disponibles. Debe evaluar los modelos primero")
            
        df_metricas = pd.DataFrame(self.metricas_modelos).round(4)
        return df_metricas.transpose()

def crear_y_evaluar_sistema(ratings_finales):
    """
    Función principal para crear y evaluar el sistema de recomendación
    """
    # Verificar que tenemos suficientes datos
    if len(ratings_finales) < 5:
        raise ValueError("Se necesitan al menos 5 ratings para crear el sistema")
        
    # Verificar que tenemos variabilidad en los ratings
    if ratings_finales['Rating'].std() < 0.1:
        raise ValueError("No hay suficiente variabilidad en los ratings")
        
    # Crear sistema
    sistema = SistemaRecomendacion(ratings_finales)
    
    # Evaluar modelos
    try:
        metricas = sistema.evaluar_modelos()
        print("\nMétricas de evaluación:")
        print(sistema.mostrar_metricas())
        
        # Entrenar mejor modelo
        sistema.entrenar_mejor_modelo()
        
    except Exception as e:
        print(f"Error en la evaluación del sistema: {str(e)}")
        raise
    
    return sistema

# Ejemplo de uso:
"""
try:
    # Crear y evaluar el sistema
    sistema = crear_y_evaluar_sistema(ratings_finales)
    
    # Generar recomendaciones para un instrumento específico
    instrumento_ejemplo = ratings_finales['Instrumento_A'].iloc[126]
    recomendaciones = sistema.generar_recomendaciones(instrumento_ejemplo, n=20)
    print("\nRecomendaciones para instrumento", instrumento_ejemplo)
    print(recomendaciones)
    
except Exception as e:
    print(f"Error: {str(e)}")
    
"""

'\ntry:\n    # Crear y evaluar el sistema\n    sistema = crear_y_evaluar_sistema(ratings_finales)\n    \n    # Generar recomendaciones para un instrumento específico\n    instrumento_ejemplo = ratings_finales[\'Instrumento_A\'].iloc[126]\n    recomendaciones = sistema.generar_recomendaciones(instrumento_ejemplo, n=20)\n    print("\nRecomendaciones para instrumento", instrumento_ejemplo)\n    print(recomendaciones)\n    \nexcept Exception as e:\n    print(f"Error: {str(e)}")\n    \n'

In [29]:
# Crear y entrenar el sistema de recomendación
sistema = SistemaRecomendacion(ratings_finales)


sistema.evaluar_modelos()

Número mínimo de ratings por instrumento: 24
Número máximo de ratings por instrumento: 502
Promedio de ratings por instrumento: 219.45
KNN configurado con k=20

Total de instrumentos únicos: 418
Total de ratings: 45866

Evaluando modelos...
✓ Modelo SVD evaluado exitosamente
  RMSE: 0.4739
  MAE: 0.3454
✓ Modelo NMF evaluado exitosamente
  RMSE: 0.4202
  MAE: 0.3018
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
✓ Modelo KNN evaluado exitosamente
  RMSE: 0.3

{'SVD': {'RMSE_medio': 0.4739296437994135,
  'MAE_medio': 0.34535022784143954,
  'tiempo_fit_medio': 0.16288356781005858,
  'tiempo_test_medio': 0.023419952392578124},
 'NMF': {'RMSE_medio': 0.4202271560499195,
  'MAE_medio': 0.30183709496084843,
  'tiempo_fit_medio': 0.2998356819152832,
  'tiempo_test_medio': 0.03406476974487305},
 'KNN': {'RMSE_medio': 0.34803610006193647,
  'MAE_medio': 0.23031622131777957,
  'tiempo_fit_medio': 0.1897134780883789,
  'tiempo_test_medio': 0.4140148639678955}}

In [30]:
sistema.entrenar_mejor_modelo()

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


## Gradio


In [36]:
recomendaciones = sistema.generar_recomendaciones(instrumento_id=3619)
recomendaciones


ValueError: Instrumento 3619 no encontrado en el conjunto de datos

# Interfaz gráfica

In [31]:
import gradio as gr
import pandas as pd

# Asegúrate de que `SistemaRecomendacion` y `data_arco` ya estén definidos
# Supón que `sistema` ya está creado y entrenado

# Convertir el DataFrame `data_arco` en un diccionario para una consulta rápida
descripcion_instrumentos = data_arco.set_index("Código")["Descripcion"].to_dict()

nombre_instrumentos = data_arco.set_index("Código")[ "Nombre del Instrumento"].to_dict()


# Función para obtener recomendaciones
def obtener_recomendaciones(instrumento_id):
    # Generar recomendaciones con el sistema de recomendación
    recomendaciones_df = sistema.generar_recomendaciones(instrumento_id, n=5)
    recomendaciones = recomendaciones_df["Instrumento_recomendado"].tolist()
    
    return recomendaciones

# Función para mostrar la descripción de un instrumento específico
def mostrar_descripcion(instrumento):
    return descripcion_instrumentos.get(instrumento, "Descripción no disponible")

# Función para mostrar el nombre de un instrumento específico
def mostrar_nombre(instrumento):
    return nombre_instrumentos.get(instrumento, "Nombre no disponible")

# Crear interfaz Gradio
with gr.Blocks() as interfaz:
    gr.Markdown("# Sistema de Recomendación de Instrumentos de Política")
    
    # Selección del instrumento principal
    instrumento_elegido = gr.Dropdown(label="Seleccione un instrumento",
                                      choices=list(sistema.ratings_finales["Instrumento_A"].unique()))

    nombre_inst_elegido =  gr.Textbox(label="Nombre del instrumento seleccionado")                                 
    # Descripción del instrumento seleccionado
    descripcion_instrumento_seleccionado = gr.Textbox(label="Descripción del instrumento seleccionado")

    # Sección de recomendaciones
    lista_recomendaciones = gr.Dropdown(label="Instrumentos recomendados")
    nombre_inst_elegido_rec =  gr.Textbox(label="Nombre del instrumento seleccionado")            
    descripcion_recomendada = gr.Textbox(label="Descripción del instrumento recomendado")

    # Función para actualizar lista de recomendaciones al seleccionar un instrumento
    def actualizar_recomendaciones(instrumento):
        recomendaciones = obtener_recomendaciones(instrumento)
        return gr.update(choices=recomendaciones)

    # Función para actualizar la descripción del instrumento seleccionado
    def actualizar_descripcion(instrumento):
        return mostrar_descripcion(instrumento)

    # Función para actualizar el nombre del instrumento seleccionado
    def actualizar_nombre(instrumento):
        return mostrar_nombre(instrumento)
    
    # Actualizar la lista de recomendaciones al seleccionar un instrumento
    instrumento_elegido.change(fn=actualizar_recomendaciones, inputs=[instrumento_elegido], outputs=[lista_recomendaciones])

    # Actualizar el nombre del instrumento elegido
    instrumento_elegido.change(fn=actualizar_nombre, inputs=[instrumento_elegido], outputs=[nombre_inst_elegido])

    # Mostrar la descripción del instrumento seleccionado
    instrumento_elegido.change(fn=actualizar_descripcion, inputs=[instrumento_elegido], outputs=[descripcion_instrumento_seleccionado])

    # Actualizar el nombre del instrumento recomendado al seleccionar uno de la lista
    lista_recomendaciones.change(fn=actualizar_nombre, inputs=[lista_recomendaciones], outputs=[nombre_inst_elegido_rec])

    # Actualizar la descripción del instrumento recomendado al seleccionar uno de la lista
    lista_recomendaciones.change(fn=mostrar_descripcion, inputs=[lista_recomendaciones], outputs=[descripcion_recomendada])

# Inicia la interfaz
if __name__ == "__main__":
    interfaz.launch()



Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
